In [ ]:
import torch
import os
import json
from transformers import AutoModelForCausalLM, AutoTokenizer
from manyqa import ManyQA
from huggingface_hub import login
from pydantic import BaseModel, Field

In [ ]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

In [ ]:
hf_token = os.getenv("HF_TOKEN") or os.getenv("HUGGINGFACE_HUB_TOKEN")
assert hf_token, "Ustaw zmienną środowiskową HF_TOKEN lub HUGGINGFACE_HUB_TOKEN z wartością hf_***"
login(token=hf_token, add_to_git_credential=False)

In [ ]:
# załadowanie modelu
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Uruchamiam na urządzeniu: {device}")

model_name = "speakleash/Bielik-11B-v2.6-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    # load_in_8bit=True,
    # load_in_4bit=True,
    dtype=torch.bfloat16,
)

In [ ]:
def decode(output: list[str]) -> str:
    out = output[0].split('|im_start|>assistant\n')[-1].replace('<|im_end|>', '')
    return out

In [ ]:
def generate(messages: list[str], max_tokens: int = 500, temperature: float = 0.01, top_p: float = 0.01) -> str:
    input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True)
    attention_mask = torch.ones_like(input_ids)
    
    model_inputs = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    
    generated_ids = model.generate(
        model_inputs, 
        attention_mask=attention_mask, 
        max_new_tokens=max_tokens, 
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
    )
    decoded = tokenizer.batch_decode(generated_ids)
    return decode(decoded)

In [ ]:
# test modelu
messages = [
    {"role": "system", "content": "Odpowiadaj krótko, precyzyjnie i wyłącznie w języku polskim."},
    {"role": "user", "content": "Jakie mamy pory roku w Polsce?"},
    # {"role": "assistant", "content": "W Polsce mamy 4 pory roku: wiosna, lato, jesień i zima."},
    # {"role": "user", "content": "Która jest najcieplejsza?"}
]

input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True)
attention_mask = torch.ones_like(input_ids)

model_inputs = input_ids.to(device)
attention_mask = attention_mask.to(device)
model.to(device)

generated_ids = model.generate(
    model_inputs, 
    attention_mask=attention_mask, 
    max_new_tokens=500, 
    do_sample=True,
    temperature=0.1,
    top_p=0.1,
)
decoded = tokenizer.batch_decode(generated_ids)
print(decode(decoded))

In [ ]:
# model odpowiedzi
class Response(BaseModel):
    # chain_of_thought: conlist(str, max_length=2) = Field(..., description="Łańcuch myśli")
    answer: str = Field(..., description='Odpowiedź na pytanie użytkownika')
    confidence: float = Field(..., description='Liczba w skali od 0 do 1 oznaczająca pewność udzielenia poprawnej odpowiedzi. 1 oznacza całkowitą pewność poprawnej odpowiedzi')

class AnswerCheck(BaseModel):
    correct: bool = Field(False, description='Flaga oznaczająca czy udzielona odpowiedź jest poprawna z kluczem')
    clarification: str = Field(..., description='Bardzo krótkie i rzeczowe wyjaśnienie decyzji')

In [ ]:
# załaduj obiekt z pytaniami
with open('QAData.json', 'r') as f:
    data = json.loads(f.read())

questions = ManyQA(**data)

In [ ]:
# test pytania
question = questions.qa[0]
print(question)

In [ ]:
# zadanie pytania
def ask_question(question) -> str:
    sys_instr = (
            "Jesteś ekspertem od historii polski\n"
            "Zwróć WYŁĄCZNIE poprawny JSON, bez komentarzy, bez ``` i bez dodatkowego tekstu. "
            "JSON musi być zgodny ze schematem: " + json.dumps(Response.model_json_schema(), ensure_ascii=False) +
            '\nPrzykład: {"answer": "63", "confidence": 0.83}' 
        )
    messages = [
        {'role': 'system', 'content': sys_instr},
        {'role': 'user', 'content': f"Pytanie: {question.question}\nOczekiwany format odpowiedzi: {question.answer_format}"}
    ]
    
    response = generate(messages)
    return response

In [ ]:
# sprawdzenie poprawności odpowiedzi
def check_asnwer(question, model_resp) -> str:
    example = '{"correct": true, "clarification": "Odpowiedź jest poprawna, ponieważ dokładnie odpowiada kluczowi."}'
    ex1 = '{"correct": true, "clarification": "Odpowiedź jest poprawna — użytkownik wskazał właściwą osobę."}'
    ex2 = '{"correct": false, "clarification": "Niepoprawna — użytkownik podał nazwę wydarzenia zamiast osoby."}'
    ex3 = '{"correct": false, "clarification": "Niepoprawna — użytkownik podał nazwę warzyw zamiast osoby (królowej Bony).}'
    ex4 = '{"correct": false, "clarification": "Niepoprawna — użytkownik podał nazwę organizacji emigracyjnej (Towarzystwo Demokratyczne Polski), a nie grupy młodych artystów (Cyganeria Warszawska).}'
    ex5 = '{"correct": false, "clarification": "Niepoprawna — użytkownik nie podał wszystkich funkcji jakie pełnił Jan Zamoyski.}'
    sys_instr = f"""ROLA: Jesteś egzaminatorem testów.
    
ZADANIE: Twoim zadaniem jest określenie, czy odpowiedź użytkownika jest **merytorycznie zgodna** z podanym kluczem (poprawną odpowiedzią).

ZASADY OCENY:
1. Odpowiedź musi znaczeniowo odpowiadać kluczowi — czyli wskazywać to samo zjawisko, osobę, miejsce, wydarzenie lub pojęcie.
2. Dopuszczalne są drobne różnice językowe (np. 'powstania listopadowego' vs 'Powstanie Listopadowe').
3. Jeśli użytkownik odpowiada nie na temat lub podaje tylko coś związanego, ale nie tożsamego z kluczem — odpowiedź jest **niepoprawna**.
4. Wyjaśnienie (clarification) ma być krótkie, jednozdaniowe i rzeczowe.
5. Dopuszczalne są odpowiedzi w innych częściach mowy. Odpowiedź: 'powstania listopadowego' jest poprawna dla klucza: 'Powstanie Litopadowe'

WYJŚCIE: Zwróć WYŁĄCZNIE poprawny JSON, bez komentarzy, bez ``` i bez dodatkowego tekstu.
JSON musi być zgodny ze schematem: {json.dumps(AnswerCheck.model_json_schema(), ensure_ascii=False)}

PRZYKŁADY:

PYTANIE: Kto był przywódcą powstania listopadowego?
KLUCZ: Piotr Wysocki
ODPOWIEDŹ: Piotr Wysocki
OCENA: {ex1}

PYTANIE: Kto był przywódcą powstania listopadowego?
KLUCZ: Piotr Wysocki
ODPOWIEDŹ: powstanie listopadowe
OCENA: {ex2}

PYTANIE: Dzięki komu na polskie stoły trafiły nieznane wcześniej warzywa zwane włoszczyzną?
KLUCZ: Królowej Bonie
ODPOWIEDŹ: włoszczyzna
OCENA: {ex3}

PYTANIE: Ośmieszali stroje salonowe i konwencje obyczajowe. Ich nieoficjalnym, własnym pismem była gazeta "Nadwiślanin". Głosili idee demokratyczno-ludowe. Mowa o przedstawicielach…
KLUCZ: Cyganerii warszawskiej
ODPOWIEDŹ: Towarzystwa Demokratycznego Polskiego
OCENA: {ex4}

PYTANIE: Jaką funkcję pełnił Jan Zamoyski w Rzeczypospolitej: Hetman wielki koronny, Kanclerz wielki koronny czy Generalny starosta krakowski? 
KLUCZ: Wszystkie wymienione
ODPOWIEDŹ: Hetman wielki koronny, Kanclerz wielki koronny
OCENA: {ex5}

PYTANIE: {question.question}
KLUCZ: {question.answer}
"""
    
    t = model_resp.split('"answer": ')[-1].split(', "')[0]
    
    user_instr = f"""ODPOWIEDŹ: {t}"""
    messages = [
            {'role': 'system', 'content': sys_instr},
            {'role': 'user', 'content': user_instr}
    ]
    return generate(messages)

In [ ]:
#przygotowanie liczników
correct_q, all_q = 0, 0
difficult_questions = []

In [ ]:
# uruchomienie testu
import time
from datetime import datetime
for question in questions.qa:
    all_q += 1
    print('\n'+ str(datetime.now()))
    print(f"Pytanie: {question.question}")
    start = time.perf_counter()
    model_response = ask_question(question)
    time_to_answer = time.perf_counter() - start
    print(model_response)
    print(f"({question.answer}) Sprawdzanie odpowiedzi...")
    correctness = check_asnwer(question, model_response)
    print(correctness)
    if '"correct": true' in correctness:
        correct_q+=1
    else:
        difficult_questions.append(question)
    tup = {
        'datetime': str(datetime.now()),
        'question': question.question,
        'model_response': model_response,
        'correctness': correctness,
        'time_to_answer': time_to_answer
    }
    with open('0experiment_bielik.jsonl', 'a') as f:
        f.write(json.dumps(tup, ensure_ascii=False))
        f.write('\n')
    print(f"Aktualny wynik: {correct_q}/{all_q} odpowiedzi poprawnych.")

In [ ]:
# Wynik pierwszej iteracji to XXX/570, czyli XX%

In [ ]:
for q in difficult_questions:
    print(q.question)